In [ ]:
import numpy as np
import pandas as pd

import requests
import sys
import os

#API
api_key = ''
api_secret = ''

def get_futures_current_price(symbol):
    url = f"https://fapi.binance.com/fapi/v1/ticker/price?symbol={symbol}"
    response = requests.get(url)
    data = response.json()
    return float(data['price'])

def send_telegram_message(message):
    bot_token = ''
    chat_id = ''
    channel_username = '@Crypto_anomalies_alert'
    url = f'https://api.telegram.org/bot{bot_token}/sendMessage'
    params = {'chat_id': channel_username, 'text': message}
    response = requests.post(url, json=params)

def fetch_kline_data(symbol): # 4hour
    endpoint = f"https://fapi.binance.com/fapi/v1/klines?symbol={symbol}&limit=240&interval=1m"
    response = requests.get(endpoint)
    return response.json() if response.status_code == 200 else None

def calculate_returns(close_prices):
    returns = np.diff(close_prices) / close_prices[:-1]
    absolute_returns = np.abs(returns)
    return absolute_returns

def check_trigger(returns, std):
    mean_return = np.mean(returns)
    std_return = np.std(returns)
    last_return = returns[-1]
    if last_return > mean_return + (std * std_return) or last_return < mean_return - (std * std_return):
        return True
    else:
        return False

# BY PRICE_CHANGE list 
# def fetch_top_crypto_close_prices():
#     binance_endpoint_24hr = "https://fapi.binance.com/fapi/v1/ticker/24hr"
#     response = requests.get(binance_endpoint_24hr)
#     binance_response_24hr = response.json()
#     filtered_response = [item for item in binance_response_24hr if item['symbol']]
#     sorted_data = sorted(filtered_response, key=lambda x: float(x['priceChangePercent']), reverse=True)
#     symbols_sorted_by_percent_change = [item['symbol'] for item in sorted_data]
#     return symbols_sorted_by_percent_change

# BY VOLUME list
def fetch_top_crypto_close_prices():
    binance_endpoint_24hr = "https://fapi.binance.com/fapi/v1/ticker/24hr"
    response = requests.get(binance_endpoint_24hr)
    binance_response_24hr = response.json()
    filtered_response = [item for item in binance_response_24hr if item['symbol']]
    sorted_data = sorted(filtered_response, key=lambda x: float(x['quoteVolume']), reverse=True)
    symbols_sorted_by_percent_change = [item['symbol'] for item in sorted_data]
    return symbols_sorted_by_percent_change

def run_job():
    for symbol in fetch_top_crypto_close_prices()[:100]:
        kline_data = fetch_kline_data(symbol)
        close_prices = [float(candle[4]) for candle in kline_data]  # Extract close prices from the Kline data
        returns = calculate_returns(close_prices)
        trigger = check_trigger(returns, std = 5)
        if trigger:
            message = f"https://www.binance.com/en/futures/{symbol}"
            send_telegram_message(message)

def main():
    while True:
        run_job()

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        # Log the error
        print(f"Fatal error: {e}. Restarting script.")
        
        # Restart the script
        os.execl(sys.executable, sys.executable, *sys.argv)
